# 🌿 Lab 2.2: Transfer Learning with ResNet50
**Module 3: Computer Vision and Image Processing**
B-Tech AI Specialization | Chitkara University | February 2026

---

## 🌾 Industry Scenario
> You have **500 images** of 5 types of plant diseases. A farmer app needs a classifier to identify diseases from phone photos. Training from scratch would take days and thousands of images. **Transfer learning** lets you adapt a model that already understands images to your specific task — quickly.

## 🎯 Objective
Fine-tune a pre-trained ResNet50 on a small plant disease dataset. Compare against training from scratch. Target: **≥80% validation accuracy in 10 epochs**.

**Time:** 120 minutes | **Mode:** Individual

---
### 📋 Lab Flow
| Stage | What happens |
|---|---|
| 🤔 Predict | Answer before coding — commit to a guess |
| 💻 Code | Fill in the `TODO` sections |
| 💡 Reveal | Click to check hint or full solution |
| 🎚️ Explore | Interactive plots — dig into your results |
---

## ⚙️ Setup — Run First

In [2]:
def reveal_button(hint_text, solution_code):
    import ipywidgets as widgets
    from IPython.display import display, HTML, Code
    out = widgets.Output()
    hint_btn = widgets.Button(description='💡 Hint', button_style='info',
        layout=widgets.Layout(width='120px', margin='4px'))
    sol_btn  = widgets.Button(description='✅ Solution', button_style='warning',
        layout=widgets.Layout(width='140px', margin='4px'))
    hide_btn = widgets.Button(description='🙈 Hide', button_style='',
        layout=widgets.Layout(width='100px', margin='4px'))
    def on_hint(b):
        with out:
            out.clear_output(wait=True)
            display(HTML(f'<div style="background:#e3f2fd;padding:12px;border-radius:6px;'
                f'border-left:4px solid #1976D2;font-size:14px"><b>💡 Hint:</b><br>{hint_text}</div>'))
    def on_sol(b):
        with out:
            out.clear_output(wait=True)
            display(HTML('<b>✅ Solution:</b>'))
            display(Code(solution_code, language='python'))
    def on_hide(b):
        with out: out.clear_output()
    hint_btn.on_click(on_hint); sol_btn.on_click(on_sol); hide_btn.on_click(on_hide)
    display(widgets.HBox([hint_btn, sol_btn, hide_btn]), out)

print("reveal_button() ready ✅")

reveal_button() ready ✅


In [ ]:
# ✏️ Your predictions (as comments):
# 1. Train: 80   Validation: 20
# 2. Validation is needed because it helps us to evaluate the model's performance on unseen data and prevents overfitting.
# 3. With only 100 images per class, the risk is that the model may not generalize well to new data, leading to overfitting. Data augmentation can help mitigate this by artificially increasing the diversity of the training data.

In [ ]:
import urllib.request, zipfile, shutil, random

# Download the flower_photos dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)

# get_file returns the cache dir; the tgz extracts into a nested flower_photos/ subfolder.
# Adjust if needed so data_dir points at the folder containing the class subdirectories.
nested = os.path.join(data_dir, 'flower_photos')
if os.path.exists(nested):
    data_dir = nested

print(f"data_dir: {data_dir}")
print(f"Contents: {sorted(os.listdir(data_dir))}")

# flower_photos has class folders directly (daisy/, roses/, etc.) — no train/val split.
# We create one here: 80% train, 20% val.
SPLIT_DIR = os.path.join(os.path.dirname(data_dir), 'flower_photos_split')
TRAIN_DIR = os.path.join(SPLIT_DIR, 'train')
VAL_DIR   = os.path.join(SPLIT_DIR, 'val')

if not os.path.exists(SPLIT_DIR):
    print("Creating train/val split...")
    random.seed(42)
    class_names = [d for d in sorted(os.listdir(data_dir))
                   if os.path.isdir(os.path.join(data_dir, d))]
    for cls in class_names:
        src_cls = os.path.join(data_dir, cls)
        images  = [f for f in os.listdir(src_cls)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)
        split   = int(0.8 * len(images))
        for split_name, subset in [('train', images[:split]), ('val', images[split:])]:
            dst = os.path.join(SPLIT_DIR, split_name, cls)
            os.makedirs(dst, exist_ok=True)
            for img in subset:
                shutil.copy(os.path.join(src_cls, img), os.path.join(dst, img))
    print("Split created ✅")
else:
    print("Split already exists ✅")

# Quick check — print class names and image counts
classes = sorted(os.listdir(TRAIN_DIR))
print(f"\nClasses found ({len(classes)}): {classes}")
for cls in classes:
    n_train = len(os.listdir(os.path.join(TRAIN_DIR, cls)))
    n_val   = len(os.listdir(os.path.join(VAL_DIR,   cls)))
    print(f"  {cls}: {n_train} train | {n_val} val")


Creating train/val split...
Split created ✅

Classes found (1): ['flower_photos']
  flower_photos: 0 train | 0 val


---
## Task 2: Data Augmentation

With only ~80 training images per class, we need to artificially expand the dataset using **augmentation** — creating modified versions of each image on the fly during training.

### 🤔 Predict First
Look at the augmentation parameters below. For each one, predict:
- What does it do visually to the image?
- Does it make sense for plant disease photos? (Would a real phone photo look like this?)

| Parameter | Your prediction | Makes sense? |
|---|---|---|
| `horizontal_flip=True` | Flips the image horizontally (left-right) | Yes, since plant photos can be taken from either side |
| `rotation_range=20` | Rotates the image by up to 20 degrees in either direction | Yes, real photos may be taken at slight angles |
| `zoom_range=0.2` | Zooms in or out by up to 20% | Yes, phone cameras may zoom in or out slightly |
| `width_shift_range=0.1` | Shifts the image horizontally by up to 10% of its width | Yes, real photos may be slightly off-center due to camera positioning |

### 💻 Build the Data Generators

In [ ]:
reveal_button(
    hint_text="Validation generator should have <b>no augmentation</b> — only <code>preprocessing_function</code>. "
              "Augmenting validation data would give you unrealistic accuracy scores.",
    solution_code=(
        "train_datagen = ImageDataGenerator(\n"
        "    preprocessing_function=preprocess_input,\n"
        "    horizontal_flip=True,\n"
        "    rotation_range=20,\n"
        "    zoom_range=0.2,\n"
        "    width_shift_range=0.1,\n"
        "    height_shift_range=0.1\n"
        ")\n\n"
        "val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)\n\n"
        "train_generator = train_datagen.flow_from_directory(\n"
        "    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'\n"
        ")\n"
        "val_generator = val_datagen.flow_from_directory(\n"
        "    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'\n"
        ")"
    )
)

In [ ]:
# Visualise augmentation — see what the model actually trains on
sample_batch, _ = next(train_generator)
sample_img_raw  = sample_batch[0]

# Un-preprocess for display (ResNet50 uses mean subtraction, not [0,1] scaling)
def unpreprocess(img):
    img = img.copy()
    img[..., 0] += 103.939
    img[..., 1] += 116.779
    img[..., 2] += 123.68
    return np.clip(img[..., ::-1] / 255.0, 0, 1)  # BGR → RGB

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("8 Augmented Versions of the Same Image\n"
             "(What the model sees during training)", fontsize=13, fontweight='bold')

aug_gen = train_datagen.flow(
    np.expand_dims(sample_batch[0], 0), batch_size=1
)
for ax in axes.flat:
    aug_img = next(aug_gen)[0]
    ax.imshow(unpreprocess(aug_img))
    ax.axis('off')

plt.tight_layout()
plt.show()
print("✏️ Observation: How different do these look from each other?")
print("   Would you expect a plant photo from a phone to look like these?")

In [ ]:
# ✏️ Your predictions:
# 1. We freeze because...
# 2. include_top=False means...
# 3. GlobalAveragePooling vs Flatten:

In [ ]:
NUM_CLASSES = 5   # adjust if your dataset has a different number

# TODO: Load ResNet50 base — no top, pretrained on ImageNet
base_model = ResNet50(
    # weights=...,
    # include_top=...,
    # input_shape=...
)

# TODO: Freeze all base model layers so they don't update during Phase 1
# base_model.trainable = ...

# TODO: Build the full model by adding a classification head
model = models.Sequential([
    base_model,
    # layers.GlobalAveragePooling2D(),
    # layers.Dense(256, activation='relu'),
    # layers.Dropout(0.5),
    # layers.Dense(NUM_CLASSES, activation='softmax'),
])

# TODO: Compile with adam and categorical_crossentropy
# model.compile(...)

# Check: how many layers are trainable?
trainable   = sum(1 for l in model.layers[0].layers if l.trainable)
untrainable = sum(1 for l in model.layers[0].layers if not l.trainable)
print(f"ResNet50 layers — Trainable: {trainable} | Frozen: {untrainable}")
model.summary()

---
## Task 4: Phase 1 — Train the Classification Head (10 epochs)

In [ ]:
reveal_button(
    hint_text="Call <code>model.fit(train_generator, epochs=10, validation_data=val_generator)</code>. "
              "Store the result in <code>history_phase1</code>.",
    solution_code=(
        "history_phase1 = model.fit(\n"
        "    train_generator,\n"
        "    epochs=10,\n"
        "    validation_data=val_generator\n"
        ")"
    )
)

---
## Task 5: Phase 2 — Fine-Tuning (Unfreeze Last 20 Layers)

Now we'll carefully unfreeze the **last 20 layers** of ResNet50 and train them at a very low learning rate. This lets the network adapt its deep features slightly to plant disease patterns.

### 🤔 Predict First
1. Why must the learning rate be **much lower** in fine-tuning (1e-5 vs 1e-3)?
2. Why do we unfreeze only the **last** layers, not the first?

In [ ]:
# ✏️ Your predictions:
# 1. Lower LR because...
# 2. Last layers because...

In [ ]:
# TODO: Unfreeze the last 20 layers of the base model
base_model = model.layers[0]  # get the ResNet50 sub-model

# Step 1: make base model trainable overall
# base_model.trainable = True

# Step 2: freeze everything EXCEPT the last 20 layers
# for layer in base_model.layers[:-20]:
#     layer.trainable = False

# TODO: Re-compile with a much lower learning rate
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=1e-5),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# Check how many are now trainable
trainable = sum(1 for l in base_model.layers if l.trainable)
print(f"Now trainable layers in ResNet50: {trainable}")

# TODO: Train for 5 more epochs
# history_phase2 = model.fit(...)

history_phase2 = None

In [ ]:
reveal_button(
    hint_text="<code>base_model.trainable = True</code> first, then loop: "
              "<code>for layer in base_model.layers[:-20]: layer.trainable = False</code>. "
              "Re-compile with <code>learning_rate=1e-5</code>.",
    solution_code=(
        "base_model = model.layers[0]\n"
        "base_model.trainable = True\n"
        "for layer in base_model.layers[:-20]:\n"
        "    layer.trainable = False\n\n"
        "model.compile(\n"
        "    optimizer=optimizers.Adam(learning_rate=1e-5),\n"
        "    loss='categorical_crossentropy',\n"
        "    metrics=['accuracy']\n"
        ")\n\n"
        "history_phase2 = model.fit(\n"
        "    train_generator, epochs=5, validation_data=val_generator\n"
        ")"
    )
)

---
## 🎚️ Task 6: Explore — Interactive Training Curves

Use the controls below to examine your training history. Look for:
- Where does Phase 1 plateau? Where does Phase 2 give an extra push?
- Is there a gap between train and val accuracy? What does that mean?
- At what epoch does the model first exceed 80% validation accuracy?

In [ ]:
# Build combined history from both phases
def build_history_dict(h1, h2):
    """Merge two History objects into one dict for plotting."""
    combined = {}
    for key in h1.history:
        p2_vals = h2.history.get(key, [])
        combined[key] = h1.history[key] + p2_vals
    combined['phase_boundary'] = len(h1.history['accuracy'])
    return combined

# ── Interactive curve explorer ────────────────────────────────────────────────
metric_toggle = widgets.ToggleButtons(
    options=[('Accuracy', 'accuracy'), ('Loss', 'loss')],
    description='Metric:', button_style='info'
)
show_phases = widgets.Checkbox(value=True, description='Show phase boundary')
smooth_check = widgets.Checkbox(value=False, description='Smooth curves')
out_plot = widgets.Output()

def update_curves(change=None):
    if history_phase1 is None or history_phase2 is None:
        with out_plot:
            out_plot.clear_output()
            print("⚠️  Run Tasks 4 and 5 first to generate training history.")
        return

    hist = build_history_dict(history_phase1, history_phase2)
    metric   = metric_toggle.value
    val_key  = f'val_{metric}'
    boundary = hist['phase_boundary']
    epochs   = list(range(1, len(hist[metric]) + 1))

    def smooth(vals, w=3):
        return [np.mean(vals[max(0,i-w):i+1]) for i in range(len(vals))]

    train_vals = smooth(hist[metric])     if smooth_check.value else hist[metric]
    val_vals   = smooth(hist[val_key])    if smooth_check.value else hist[val_key]

    with out_plot:
        out_plot.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 5))

        ax.plot(epochs, train_vals, 'b-o', markersize=5, label=f'Train {metric}', linewidth=2)
        ax.plot(epochs, val_vals,   'r-o', markersize=5, label=f'Val {metric}',   linewidth=2)

        if show_phases.value:
            ax.axvline(x=boundary + 0.5, color='purple', linestyle='--', alpha=0.7, linewidth=2)
            ymin, ymax = ax.get_ylim()
            ax.text(boundary * 0.5, ymax * 0.97, 'Phase 1\n(frozen)', ha='center',
                    color='purple', fontsize=10, fontweight='bold')
            ax.text(boundary + (len(epochs) - boundary) * 0.5, ymax * 0.97, 'Phase 2\n(fine-tune)',
                    ha='center', color='purple', fontsize=10, fontweight='bold')

        if metric == 'accuracy':
            ax.axhline(y=0.80, color='green', linestyle=':', alpha=0.8, linewidth=1.5,
                       label='80% target')
            best_val  = max(val_vals)
            best_ep   = val_vals.index(best_val) + 1
            ax.annotate(f'Best: {best_val:.1%} @ ep{best_ep}',
                        xy=(best_ep, best_val), xytext=(best_ep + 1, best_val - 0.05),
                        arrowprops=dict(arrowstyle='->', color='red'), color='red', fontsize=10)

        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel(metric.capitalize(), fontsize=12)
        ax.set_title(f'Training Curves — Transfer Learning with ResNet50', fontsize=13, fontweight='bold')
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

metric_toggle.observe(update_curves, names='value')
show_phases.observe(update_curves, names='value')
smooth_check.observe(update_curves, names='value')

display(widgets.VBox([
    widgets.HBox([metric_toggle, show_phases, smooth_check]),
    out_plot
]))
update_curves()

---
## ✍️ Reflection

Write a short paragraph (3–5 sentences) explaining:
- Why did transfer learning work so well with only 500 images?
- What did Phase 1 learn vs Phase 2?
- Would you expect the same result if you trained from scratch? Why?

In [ ]:
# ✏️ Your reflection:
# Transfer learning worked well because...
# In Phase 1, the model learned...
# In Phase 2, fine-tuning added...
# Training from scratch would have...